<a href="https://colab.research.google.com/github/s-u-s-h-ii/Text-to-Video/blob/main/Main_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from diffusers import DiffusionPipeline
from gtts import gTTS
import moviepy.editor as mp

base_dir = "/content"
text = 'In the vast expanse of the cosmos, two galactic empires clashed relentlessly. Laser beams pierced through the void, illuminating the darkness with their deadly dance. Planetary alliances trembled as the conflict escalated to unprecedented levels. Starships of immense power engaged in a ballet of destruction, their hulls gleaming with the scars of battle. Amidst the chaos, a courageous squadron led the charge, their resolve unyielding against the onslaught of enemy forces.'
prompt_sentences = text.split('.')


# Initialize the Diffusion Pipeline
pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# Create directories to store generated files
image_dir = os.path.join(base_dir, "images")
audio_dir = os.path.join(base_dir, "audio")
short_video_dir = os.path.join(base_dir, "short_videos")
final_video_dir = os.path.join(base_dir, "final_video")
os.makedirs(image_dir, exist_ok=True)
os.makedirs(audio_dir, exist_ok=True)
os.makedirs(short_video_dir, exist_ok=True)
os.makedirs(final_video_dir, exist_ok=True)

# Generate images and corresponding audio for each sentence
for i, sentence in enumerate(prompt_sentences):
    # Generate image
    image = pipe(sentence).images[0]
    image_path = os.path.join(image_dir, f"image_{i}.png")
    image.save(image_path)

    # Convert text to speech
    tts = gTTS(sentence)
    audio_path = os.path.join(audio_dir, f"audio_{i}.mp3")
    tts.save(audio_path)

# Convert each image to a short video clip
for i in range(len(prompt_sentences)):
    image_path = os.path.join(image_dir, f"image_{i}.png")
    video_clip_path = os.path.join(short_video_dir, f"video_{i}.mp4")

    # Create a short video clip from the image
    clip = mp.ImageClip(image_path, duration=2)  # Adjust duration as needed
    clip.write_videofile(video_clip_path, fps=24)

# Combine images with audio into a video
video_clips = []
for i in range(len(prompt_sentences)):
    video_clip_path = os.path.join(short_video_dir, f"video_{i}.mp4")
    audio_path = os.path.join(audio_dir, f"audio_{i}.mp3")
    video_clip = mp.VideoFileClip(video_clip_path)
    audio_clip = mp.AudioFileClip(audio_path)
    video_clip = video_clip.set_audio(audio_clip)
    video_clips.append(video_clip)

final_video = mp.concatenate_videoclips(video_clips, method="compose")
final_video_path = os.path.join(final_video_dir, "final_video.mp4")
final_video.write_videofile(final_video_path, codec="libx264", fps=24)

print(f"Saved final video to {final_video_path}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model_index.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

...

Moviepy - Done !
Moviepy - video ready /content/short_videos/video_1.mp4
Moviepy - Building video /content/short_videos/video_2.mp4.
Moviepy - Writing video /content/short_videos/video_2.mp4



Moviepy - Done !
Moviepy - video ready /content/short_videos/video_2.mp4
Moviepy - Building video /content/short_videos/video_3.mp4.
Moviepy - Writing video /content/short_videos/video_3.mp4



Moviepy - Done !
Moviepy - video ready /content/short_videos/video_3.mp4
Moviepy - Building video /content/final_video/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/final_video/final_video.mp4



Moviepy - Done !
Moviepy - video ready /content/final_video/final_video.mp4
Saved final video to /content/final_video/final_video.mp4
